In [2]:
# Step 1: Read and process 10,000 .txt files

import os
import pandas as pd

def try_multiple_delimiters(file_path):
    """
    Attempts to read a file using different delimiters.
    """
    delimiters = ['\t', ',', ';']  # Common delimiters to try
    for delimiter in delimiters:
        try:
            df = pd.read_csv(file_path, sep=delimiter, encoding='utf-8', on_bad_lines='skip', dtype={'PARTICIPANT_ID': str})
            # If we get more than one column, we assume we have the right delimiter
            if df.shape[1] > 1:
                return df
        except Exception as e:
            pass  # Try the next delimiter
    raise ValueError("Could not determine delimiter")

def read_keystroke_data(samples_dir, output_csv='all_samples_combined1.csv', start_batch=1, end_batch=100):
    """
    Reads keystroke data from multiple subfolders, handles errors, and saves valid data to CSV.

    Parameters:
    - samples_dir (str): The directory where the 'samples' folder resides, which contains multiple subfolders with .txt files.
    - output_csv (str): The output path for the CSV file to save valid data.
    - start_batch (int): The starting batch number (inclusive).
    - end_batch (int): The ending batch number (inclusive).

    Returns:
    - A pandas DataFrame containing valid keystroke data.
    """
    all_data = []  # To hold data from all valid files
    total_files_processed = 0  # Counter to keep track of the total number of files processed
    total_rows_processed = 0  # Counter for the total number of rows across all files
    skipped_files = 0  # Counter for the number of skipped files
    malformed_rows = 0  # Counter for the number of malformed rows

    # Expected columns in the file
    required_columns = ['PARTICIPANT_ID', 'TEST_SECTION_ID', 'SENTENCE', 'USER_INPUT',
                        'KEYSTROKE_ID', 'PRESS_TIME', 'RELEASE_TIME', 'LETTER', 'KEYCODE']

    # Traverse each subfolder (e.g., samples_01, samples_02, ..., samples_100)
    for batch_num in range(start_batch, end_batch + 1):
        subdir = f'samples_{batch_num:02d}'
        subdir_path = os.path.join(samples_dir, subdir)

        if os.path.isdir(subdir_path):
            # Traverse each .txt file in the subfolder
            for file_name in os.listdir(subdir_path):
                if file_name.endswith('.txt'):
                    file_path = os.path.join(subdir_path, file_name)
                    total_files_processed += 1  # Increment the counter for each file

                    print(f"Processing file: {file_path}")  # Debugging statement

                    try:
                        # Try reading the file using multiple delimiters
                        df = try_multiple_delimiters(file_path)

                        # Log the number of rows in the current file
                        print(f"File {file_name} has {df.shape[0]} rows")

                        # Ensure required columns exist
                        if not all(col in df.columns for col in required_columns):
                            print(f"Missing columns in {file_path}")
                            skipped_files += 1
                            continue  # Skip this file if it doesn't have the required columns

                        # If the number of columns is 1, it means that the file might be incorrectly formatted
                        if len(df.columns) == 1:
                            print(f"Malformed data in {file_path}, skipping.")
                            skipped_files += 1
                            continue  # Skip files with misformatted rows

                        # Filter out rows where all values are under one column
                        malformed_df = df[df.apply(lambda row: row.count() == 1, axis=1)]
                        malformed_rows += len(malformed_df)
                        df = df[df.apply(lambda row: row.count() > 1, axis=1)]

                        # Append the number of rows to the total rows processed
                        total_rows_processed += df.shape[0]

                        # Append the valid DataFrame to the list
                        all_data.append(df)

                    except Exception as e:
                        print(f"Error processing file {file_path}: {e}")
                        skipped_files += 1
                        continue  # Skip files with errors like encoding issues or missing data

    # Concatenate all valid data into one DataFrame
    if not all_data:
        print("No valid data found.")
        return pd.DataFrame()  # Return an empty DataFrame if no valid data was found

    full_data = pd.concat(all_data, ignore_index=True)

    # Verify the number of rows before saving to CSV
    print(f"Total rows in concatenated DataFrame: {full_data.shape[0]}")
    print(f"Total malformed rows removed: {malformed_rows}")

    # Save the data to CSV
    full_data.to_csv(output_csv, index=False)
    print(f"Valid data saved to {output_csv}")

    # Print the total number of files processed and total rows processed
    print(f"Total number of .txt files processed: {total_files_processed}")
    print(f"Total number of rows processed: {total_rows_processed}")
    print(f"Total number of skipped files: {skipped_files}")

    return full_data

# Example usage:
samples_directory = "samples"  # Replace with your actual path
read_keystroke_data(samples_directory, output_csv='demographics_csv/part1_uncleaned_all.csv', start_batch=1, end_batch=1)

Processing file: samples\samples_01\100001_keystrokes.txt
File 100001_keystrokes.txt has 658 rows
Processing file: samples\samples_01\100003_keystrokes.txt
File 100003_keystrokes.txt has 806 rows
Processing file: samples\samples_01\100007_keystrokes.txt
File 100007_keystrokes.txt has 801 rows
Processing file: samples\samples_01\100008_keystrokes.txt
File 100008_keystrokes.txt has 687 rows
Processing file: samples\samples_01\100013_keystrokes.txt
File 100013_keystrokes.txt has 744 rows
Processing file: samples\samples_01\100016_keystrokes.txt
File 100016_keystrokes.txt has 776 rows
Processing file: samples\samples_01\10001_keystrokes.txt
File 10001_keystrokes.txt has 763 rows
Processing file: samples\samples_01\100020_keystrokes.txt
File 100020_keystrokes.txt has 649 rows
Processing file: samples\samples_01\100030_keystrokes.txt
File 100030_keystrokes.txt has 654 rows
Processing file: samples\samples_01\100031_keystrokes.txt
File 100031_keystrokes.txt has 545 rows
Processing file: sampl

,PARTICIPANT_ID,TEST_SECTION_ID,SENTENCE,USER_INPUT,KEYSTROKE_ID,PRESS_TIME,RELEASE_TIME,LETTER,KEYCODE
0,100001,1090979,Was wondering if you and Natalie connected?,Was wondering if you and Natalie connected?,51891207.0,1.473275e+12,1473275372663,SHIFT,16
1,100001,1090979,Was wondering if you and Natalie connected?,Was wondering if you and Natalie connected?,51891214.0,1.473275e+12,1473275372703,W,87
2,100001,1090979,Was wondering if you and Natalie connected?,Was wondering if you and Natalie connected?,51891219.0,1.473275e+12,1473275372903,a,65
3,100001,1090979,Was wondering if you and Natalie connected?,Was wondering if you and Natalie connected?,51891226.0,1.473275e+12,1473275372975,s,83
4,100001,1090979,Was wondering if you and Natalie connected?,Was wondering if you and Natalie connected?,51891231.0,1.473275e+12,1473275373079,,32
...,...,...,...,...,...,...,...,...,...
71456,1002,10502,Thursday works better for me.,Thursday works better for me.,502839.0,1.471949e+12,1471948648921,r,82
71457,1002,10502,Thursday works better for me.,Thursday works better for me.,502843.0,1.471949e+12,1471948648973,,32
71458,1002,10502,Thursday works better for me.,Thursday works better for me.,502846.0,1.471949e+12,1471948649104,m,77
71459,1002,10502,Thursday works better for me.,Thursday works better for me.,502858.0,1.471949e+12,1471948649190,e,69


In [3]:
# Step 2: Clean the CSV file

import pandas as pd

def clean_csv(input_csv, output_csv):
    """
    Cleans a CSV file by removing rows where more than one column has NaN or <unset> values.

    Parameters:
    - input_csv (str): The path to the input CSV file.
    - output_csv (str): The path to the output CSV file to save cleaned data.
    """
    # Read the CSV file into a DataFrame
    df = pd.read_csv(input_csv, sep=',')

    # Define a function to check for NaN or <unset> values
    def is_unset_or_nan(value):
        return pd.isna(value) or value == '<unset>'

    # Identify and filter out rows where more than one column has NaN or <unset> values
    malformed_rows = df[df.apply(lambda row: sum(is_unset_or_nan(val) for val in row) > 1, axis=1)]
    cleaned_df = df[df.apply(lambda row: sum(is_unset_or_nan(val) for val in row) <= 1, axis=1)]

    # Log the number of malformed rows removed
    print(f"Total malformed rows removed: {len(malformed_rows)}")
    print("Malformed rows:")
    print(malformed_rows)

    # Save the cleaned DataFrame to a new CSV file
    cleaned_df.to_csv(output_csv, index=False)
    print(f"Cleaned data saved to {output_csv}")

# Example usage:
input_csv_path = 'demographics_csv/part1_uncleaned_all.csv'  # Replace with your actual input CSV path
output_csv_path = 'demographics_csv/part1_cleaned_samples_combined.csv'  # Replace with your desired output CSV path
clean_csv(input_csv_path, output_csv_path)

Total malformed rows removed: 0
Malformed rows:
Empty DataFrame
Columns: [PARTICIPANT_ID, TEST_SECTION_ID, SENTENCE, USER_INPUT, KEYSTROKE_ID, PRESS_TIME, RELEASE_TIME, LETTER, KEYCODE]
Index: []
Cleaned data saved to demographics_csv/part1_cleaned_samples_combined.csv


In [1]:
import pandas as pd
import numpy as np

def calculate_new_columns(group):
    # Create the new columns based on transformations of the KEYPRESS_ID
    group['D1U1'] = group['KEYSTROKE_ID'] * 1.1
    group['D1U2'] = group['KEYSTROKE_ID'] * 1.2
    group['D1U3'] = group['KEYSTROKE_ID'] * 1.3
    group['D1D2'] = group['KEYSTROKE_ID'] * 1.4
    group['D1D3'] = group['KEYSTROKE_ID'] * 1.5
    group['U1D2'] = group['D1U1'] - group['D1D2']
    
    # Calculate Z-scores for each of the columns
    for col in ['D1U1', 'D1U2', 'D1U3', 'D1D2', 'D1D3', 'U1D2']:
        group[f'{col}_Z_SCORE'] = (group[col] - group[col].mean()) / group[col].std()
    
    return group

def calculate_mean_values(df):
    # Calculate mean values for the columns
    mean_values = df.groupby('PARTICIPANT_ID').agg({
        'D1U1': 'mean',
        'D1U2': 'mean',
        'D1U3': 'mean',
        'D1D2': 'mean',
        'D1D3': 'mean',
        'U1D2': 'mean'
    }).reset_index()

    # Calculate the mean of Z-scores for each column
    z_score_columns = ['D1U1_Z_SCORE', 'D1U2_Z_SCORE', 'D1U3_Z_SCORE', 'D1D2_Z_SCORE', 'D1D3_Z_SCORE', 'U1D2_Z_SCORE']
    z_score_mean = df.groupby('PARTICIPANT_ID')[z_score_columns].mean().reset_index()

    # Merge the mean values with the Z_SCORE mean values
    mean_values = pd.merge(mean_values, z_score_mean, on='PARTICIPANT_ID')

    # Renaming the columns for clarity
    mean_values.columns = [
        'PARTICIPANT_ID', 'D1U1_MEAN', 'D1U2_MEAN', 'D1U3_MEAN',
        'D1D2_MEAN', 'D1D3_MEAN', 'U1D2_MEAN',
        'D1U1_Z_SCORE_MEAN', 'D1U2_Z_SCORE_MEAN', 'D1U3_Z_SCORE_MEAN',
        'D1D2_Z_SCORE_MEAN', 'D1D3_Z_SCORE_MEAN', 'U1D2_Z_SCORE_MEAN'
    ]
    
    return mean_values

def process_keystroke_data(input_csv, output_csv):
    df = pd.read_csv(input_csv)
    
    # Apply the calculation to each participant group
    df = df.groupby('PARTICIPANT_ID').apply(calculate_new_columns)
    
    # Calculate the mean values and mean Z-scores
    mean_values = calculate_mean_values(df)
    
    # Save the final results to a CSV file
    mean_values.to_csv(output_csv, index=False)
    print(f"Final data saved to {output_csv}")

# Example usage:
input_csv_path = 'demographics_csv/part1_cleaned_samples_combined.csv'  # Replace with your actual input CSV path
output_csv_path = 'demographics_csv/final5_keystroke_features.csv'  # Replace with your desired output CSV path
process_keystroke_data(input_csv_path, output_csv_path)


Final data saved to demographics_csv/final5_keystroke_features.csv


In [1]:
import pandas as pd

def merge_with_metadata(keystroke_csv, metadata_csv, output_csv):
    # Read the input CSV files
    keystroke_df = pd.read_csv(keystroke_csv)
    metadata_df = pd.read_csv(metadata_csv)

    # Merge the DataFrames on PARTICIPANT_ID
    merged_df = pd.merge(keystroke_df, metadata_df, on='PARTICIPANT_ID')

    # Save the merged DataFrame to a new CSV file
    merged_df.to_csv(output_csv, index=False)
    print(f"Merged data saved to {output_csv}")

# Example usage:
keystroke_csv_path = 'demographics_csv/final5_keystroke_features.csv'  # Replace with your actual keystroke CSV path
metadata_csv_path = 'big_keystroke_data/metadata_participants.csv'  # Replace with your actual metadata CSV path
output_csv_path = 'demographics_csv/metadata_extended.csv'  # Replace with your desired output CSV path
merge_with_metadata(keystroke_csv_path, metadata_csv_path, output_csv_path)

Merged data saved to demographics_csv/metadata_extended.csv


In [ ]:
import os
import pandas as pd

def merge_all_csv_files(input_folder, output_csv):
    # List to hold all DataFrames
    all_dataframes = []

    # Iterate over all files in the input folder
    for file_name in os.listdir(input_folder):
        if file_name.endswith('.csv'):
            file_path = os.path.join(input_folder, file_name)
            df = pd.read_csv(file_path)
            all_dataframes.append(df)

    # Concatenate all DataFrames
    combined_df = pd.concat(all_dataframes, ignore_index=True)

    # Save the combined DataFrame to a new CSV file
    combined_df.to_csv(output_csv, index=False)
    print(f"All data merged and saved to {output_csv}")

# Example usage:
input_folder_path = 'path/to/your/folder'  # Replace with your actual folder path
output_csv_path = 'metadata_master.csv'  # Replace with your desired output CSV path
merge_all_csv_files(input_folder_path, output_csv_path)